In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-02-28"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12540,2024-02-28,Brasil Nbb,19:30,União Corinthians,Unifacisa,2.88,1.38,160.5,1.82,1.88,4.5,2.04,1.24,WtzMTy5e,0.347222,0.724638,0.549451,0.531915,0.071860,0.4,1.0,NaN,NaN,285.916,236.320253,0.826537,104.742,9.329519,0.089071,181.414,244.222,98.55,206.48,0.0,0.0,0.0,0.0,0.497963,0.022933,0.344930,-2.02,-0.404,-4.653465,0.487940,0.3,-0.187940,-3.68,-0.736,-0.516304,0.475036,0.2,-0.275036
12541,2024-02-28,Eua Nba,21:30,Toronto Raptors,Dallas Mavericks,2.21,1.71,241.5,1.81,2.00,1.5,2.02,1.59,zB6Q2yT4,0.452489,0.584795,0.552486,0.500000,0.037284,0.2,0.4,NaN,NaN,232.428,90.356352,0.388750,186.160,59.136160,0.317663,232.428,230.480,215.38,268.94,0.0,0.0,0.0,0.0,0.180384,0.070525,0.168452,-1.56,-0.312,-3.878205,0.493015,0.3,-0.193015,0.27,0.054,13.148148,0.473789,0.5,0.026211
12542,2024-02-28,Eua Nba,22:00,Chicago Bulls,Cleveland Cavaliers,2.51,1.57,218.5,1.80,2.01,3.5,1.97,1.37,6TTwblWF,0.398406,0.636943,0.555556,0.497512,0.035349,0.8,0.0,NaN,NaN,330.308,117.957873,0.357115,154.782,19.384751,0.125239,265.192,190.950,411.04,146.47,1.0,0.0,0.0,0.0,0.325824,0.077949,0.254050,-1.15,-0.230,-6.565217,0.561558,0.6,0.038442,2.06,0.412,1.383495,0.560585,0.7,0.139415
12543,2024-02-28,Coreia Do Sul Wkbl Feminina,07:00,Samsung Blue Minx F,Busan BNK Sum F,2.23,1.62,135.5,1.82,1.84,1.5,2.04,1.48,llXRnOiG,0.448430,0.617284,0.549451,0.543478,0.065714,0.6,0.4,NaN,NaN,181.300,89.484001,0.493569,262.752,133.419890,0.507779,149.144,276.004,178.21,450.00,0.0,0.0,0.0,0.0,0.224070,0.007728,0.224989,-2.06,-0.412,-2.985437,0.445325,0.4,-0.045325,-5.00,-1.000,-0.620000,0.361846,0.1,-0.261846
12544,2024-02-28,Eua Ncaa,00:00,Fresno State,Utah State,5.40,1.18,141.5,1.80,1.95,9.5,2.00,1.02,hxfVUJ6B,0.185185,0.847458,0.555556,0.512821,0.032643,0.6,0.8,NaN,NaN,233.480,155.604597,0.666458,116.148,19.103719,0.164477,170.598,180.374,174.85,143.55,0.0,0.0,0.0,0.0,0.906988,0.056569,0.458917,-1.70,-0.340,-12.941176,0.568608,0.5,-0.068608,-1.49,-0.298,-0.604027,0.535273,0.7,0.164727
12545,2024-02-28,Eua Ncaa,00:30,Colorado State,Nevada,1.32,3.57,140.5,1.80,1.95,-7.5,1.95,8.00,SrfQOH2h,0.757576,0.280112,0.555556,0.512821,0.037688,0.4,0.4,NaN,NaN,139.830,67.924283,0.485763,145.942,41.163437,0.282053,106.692,163.888,113.25,99.12,0.0,0.0,0.0,0.0,0.650712,0.056569,0.859899,2.09,0.418,0.765550,0.752446,0.9,0.147554,0.97,0.194,13.247423,0.549654,0.7,0.150346
12546,2024-02-28,Europa Euroliga Feminina,16:00,Zaragoza F,CBK Mersin F,2.15,1.69,133.5,1.88,1.90,1.5,1.98,1.52,ShPU4T53,0.465116,0.591716,0.531915,0.526316,0.056832,0.4,0.2,NaN,NaN,293.492,345.557693,1.177401,109.244,26.608361,0.243568,109.792,140.444,105.60,110.20,0.0,0.0,0.0,0.0,0.169411,0.007483,0.185868,3.28,0.656,1.753049,0.000000,0.0,0.000000,0.03,0.006,115.000000,0.000000,0.0,0.000000
12547,2024-02-28,Filipinas Copa Das Filipinas,08:30,TNT Tropang Giga,Rain Or Shine Paint.,1.94,1.81,207.5,1.80,1.92,1.5,1.82,1.66,hr98wTqj,0.515464,0.552486,0.555556,0.520833,0.067950,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.049026,0.045620,0.065021,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12548,2024-02-28,Noruega Blno,14:30,Froya,Gimle,2.10,1.68,162.5,1.86,1.84,1.5,1.96,1.53,6cBG6RdO,0.476190,0.595238,0.537634,0.543478,0.071429,0.6,0.4,NaN,NaN,164.530,112.699033,0.684976,124.780,59.144878,0.473993,112.642,136.872,149.15,229.08,0.0,0.0,0.0,0.0,0.157135,0.007644,0.174244,-0.23,-0.046,-23.913043

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:30,Eua Ncaa,Colorado State,Nevada,140.5,1.80,1.0,Over
1,16:00,Polônia 1. Liga,Poznań,Starogard Gdanski,159.5,1.85,1.0,Over
2,15:30,Sérvia Primeira Liga,Sloga,Cacak 94,157.5,1.82,1.0,Over
3,21:00,Eua Ncaa,Citadel,East Tennessee St,134.5,1.91,1.0,Over
4,21:00,Eua Ncaa,Cleveland State,Wisc. Green Bay,137.5,1.80,1.0,Over
5,21:00,Eua Ncaa,Georgia Southern,Marshall,154.5,1.91,1.0,Over
6,21:00,Eua Ncaa,Jacksonville,Stetson,136.5,1.91,1.0,Over
7,21:00,Eua Ncaa,Navy,Lafayette,130.5,1.80,1.0,Over
8,21:00,Eua Ncaa,North Florida,Florida Gulf Coast,143.5,1.80,1.0,Over
9,21:00,Eua Ncaa,USC Upstate,Presbyterian,140.5,1.91,1.0,Over
